<div>
<img src="https://camo.githubusercontent.com/473dd9f992924d27457650251786464f72e54121ac6e9210add0f483ca849277/68747470733a2f2f692e696d6775722e636f6d2f3765523750616e2e706e67" width="40%">  
</div>

# Distributed Bloom for Text Generation using Prompt Tuning

In this example, we show how to use [prompt tuning](https://aclanthology.org/2021.emnlp-main.243.pdf) to adapt a test 6B version of the [BLOOM](https://huggingface.co/bigscience/bloom) model for a specific downstream task. We will run this model in a decentralized fashion using [Petals](https://github.com/bigscience-workshop/petals). Petals servers will maintain the BLOOM blocks (they are kept unchanged during adaptation), and the gradient descent will learn a few prefix tokens stored on a Petals client.

We will adapt the BLOOM model for the chatbot task using the [Personachat](https://huggingface.co/datasets/bavard/personachat_truecased) dataset. For a given dialogue context, the model has to provide a relevant answer.

First, we have to prepare all dependencies.

In [20]:
import os
import sys
sys.path.insert(0, "../../../petals")
 
import torch
import transformers
import wandb
from datasets import load_dataset
from tqdm import tqdm
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import get_scheduler
from torch import nn

# Import a Petals model
from src.client.remote_model import DistributedBloomForCausalLM

Let's set some hyperparameters for training:

In [2]:
MODEL_NAME = "bigscience/test-bloomd-6b3" # select model you like
INITIAL_PEERS = ["/ip4/193.106.95.184/tcp/31000/p2p/QmSg7izCDtowVTACbUmWvEiQZNY4wgCQ9T9Doo66K59X6q"] # add your peers adresses here, like "/ip4/192.168.1.2/tcp/31000/p2p/Qma...."
NUM_PREFIX_TOKENS = 16
DEVICE = 'cpu'
BATCH_SIZE = 4
LR = 1e-2
WEIGHT_DECAY = 0.0
NUM_SAMPLES = 1000
SEED = 42
MODEL_MAX_LENGTH = 256
TUNING_MODE = 'ptune' # choose between ['ptune', 'deep_ptune'] 

Prepare tokenizer and distributed model, connect it to servers.

In [3]:
tokenizer = transformers.BloomTokenizerFast.from_pretrained(MODEL_NAME)
tokenizer.padding_side = 'right'
tokenizer.model_max_length = MODEL_MAX_LENGTH
model = DistributedBloomForCausalLM.from_pretrained(
    MODEL_NAME, 
    initial_peers=INITIAL_PEERS, 
    pre_seq_len=NUM_PREFIX_TOKENS, 
    tuning_mode=TUNING_MODE
).to(DEVICE)

Oct 28 12:56:03.927 [WARN] [/home/jagiljazev/personalized-chat-bot/notebooks/gilyazev/../../../petals/src/client/remote_sequential.py.__init__:34] RemoteSequential is in active development; expect adventures
Some weights of DistributedBloomForCausalLM were not initialized from the model checkpoint at bigscience/test-bloomd-6b3 and are newly initialized: ['lm_head.word_embeddings.weight', 'prompt_embeddings.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Let's prepare the Personachat dataset. We need two mapping functions, one to concatenate history and candidate answers, and another for tokenization.

In [4]:
dataset = load_dataset("bavard/personachat_truecased")


def chunking(examples):
    inputs = [
        "\n-----\n".join(history) + "\n-----\n" + candidate
        for history, candidates in zip(examples["history"], examples["candidates"])
        for candidate in candidates
    ]
    return {"chunks": inputs}


def tokenize(examples):
    outputs = {
        "input_ids": tokenizer(examples["chunks"], padding='max_length', truncation=True)["input_ids"]
    }
    outputs["labels"] = outputs["input_ids"]
    return outputs


tokenized_datasets = (
    dataset
        .map(chunking, batched=True, remove_columns=dataset["train"].column_names)
        .map(tokenize, batched=True, remove_columns=["chunks"])
)


tokenized_datasets.set_format("torch")
train_dataset = tokenized_datasets["train"].shuffle(seed=SEED)
train_dataloader = DataLoader(
    train_dataset.select(list(range(NUM_SAMPLES))),
    shuffle=True,
    batch_size=BATCH_SIZE,
    drop_last=True,
)

Oct 28 12:57:16.929 [WARN] [datasets.builder._create_builder_config:427] No config specified, defaulting to: personachat_truecased/full
Oct 28 12:57:16.987 [WARN] [datasets.builder.download_and_prepare:739] Found cached dataset personachat_truecased (/home/jagiljazev/.cache/huggingface/datasets/bavard___personachat_truecased/full/1.0.0/73ee8f1a0d9e42255af5a8301877a2f3ac638e55b1cd9cbccca5ab7e23d2b638)


  0%|          | 0/2 [00:00<?, ?it/s]

Oct 28 12:57:17.053 [WARN] [datasets.arrow_dataset._map_single:2793] Loading cached processed dataset at /home/jagiljazev/.cache/huggingface/datasets/bavard___personachat_truecased/full/1.0.0/73ee8f1a0d9e42255af5a8301877a2f3ac638e55b1cd9cbccca5ab7e23d2b638/cache-5ecae882ebbd418d.arrow
Oct 28 12:57:21.147 [WARN] [datasets.arrow_dataset._map_single:2793] Loading cached processed dataset at /home/jagiljazev/.cache/huggingface/datasets/bavard___personachat_truecased/full/1.0.0/73ee8f1a0d9e42255af5a8301877a2f3ac638e55b1cd9cbccca5ab7e23d2b638/cache-7000c64a1a527e4d.arrow
Oct 28 12:57:21.478 [WARN] [datasets.arrow_dataset._map_single:2793] Loading cached processed dataset at /home/jagiljazev/.cache/huggingface/datasets/bavard___personachat_truecased/full/1.0.0/73ee8f1a0d9e42255af5a8301877a2f3ac638e55b1cd9cbccca5ab7e23d2b638/cache-76265556d7dc8064.arrow
Oct 28 12:57:38.196 [WARN] [datasets.arrow_dataset._map_single:2793] Loading cached processed dataset at /home/jagiljazev/.cache/huggingface/d

Before setting up optimizers, check the model parameters that will be trained.

In [5]:
for n, p in model.named_parameters():
    if p.requires_grad:
        print(n, p.requires_grad, p.device)

transformer.prompt_embeddings.weight True cpu


The optimizer will only work on **prompts**, they are only trainable parameters. Let's initialize optimizer and learning rate scheduler.

In [6]:
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader)
)

Let's initialize wandb for logging and start the training loop!

In [18]:
for batch in tqdm(train_dataloader):
    batch = {k: v.to(DEVICE) for k, v in batch.items()}
    print(batch)
    break

  0%|          | 0/250 [00:00<?, ?it/s]

{'input_ids': tensor([[  1270, 158204,  84148,  ...,      3,      3,      3],
        [ 47569,   1130,  25008,  ...,      3,      3,      3],
        [  1270, 158204,  84148,  ...,      3,      3,      3],
        [  1270, 158204,  84148,  ...,      3,      3,      3]]), 'labels': tensor([[  1270, 158204,  84148,  ...,      3,      3,      3],
        [ 47569,   1130,  25008,  ...,      3,      3,      3],
        [  1270, 158204,  84148,  ...,      3,      3,      3],
        [  1270, 158204,  84148,  ...,      3,      3,      3]])}


In [14]:
dataset.map(chunking, batched=True, remove_columns=dataset["train"].column_names)

Oct 28 14:52:25.194 [WARN] [datasets.arrow_dataset._map_single:2793] Loading cached processed dataset at /home/jagiljazev/.cache/huggingface/datasets/bavard___personachat_truecased/full/1.0.0/73ee8f1a0d9e42255af5a8301877a2f3ac638e55b1cd9cbccca5ab7e23d2b638/cache-5ecae882ebbd418d.arrow
Oct 28 14:52:25.244 [WARN] [datasets.arrow_dataset._map_single:2793] Loading cached processed dataset at /home/jagiljazev/.cache/huggingface/datasets/bavard___personachat_truecased/full/1.0.0/73ee8f1a0d9e42255af5a8301877a2f3ac638e55b1cd9cbccca5ab7e23d2b638/cache-7000c64a1a527e4d.arrow


DatasetDict({
    train: Dataset({
        features: ['chunks'],
        num_rows: 2628760
    })
    validation: Dataset({
        features: ['chunks'],
        num_rows: 156020
    })
})

In [16]:
dataset.map(chunking, batched=True, remove_columns=dataset["train"].column_names)['train']['chunks']

Oct 28 14:52:43.547 [WARN] [datasets.arrow_dataset._map_single:2793] Loading cached processed dataset at /home/jagiljazev/.cache/huggingface/datasets/bavard___personachat_truecased/full/1.0.0/73ee8f1a0d9e42255af5a8301877a2f3ac638e55b1cd9cbccca5ab7e23d2b638/cache-5ecae882ebbd418d.arrow
Oct 28 14:52:43.596 [WARN] [datasets.arrow_dataset._map_single:2793] Loading cached processed dataset at /home/jagiljazev/.cache/huggingface/datasets/bavard___personachat_truecased/full/1.0.0/73ee8f1a0d9e42255af5a8301877a2f3ac638e55b1cd9cbccca5ab7e23d2b638/cache-7000c64a1a527e4d.arrow


["Hi, how are you doing? I'm getting ready to do some cheetah chasing to stay in shape.\n-----\nMy mom was single with 3 boys, so we never left the projects.",
 "Hi, how are you doing? I'm getting ready to do some cheetah chasing to stay in shape.\n-----\nI try to wear all black every day. It makes me feel comfortable.",
 "Hi, how are you doing? I'm getting ready to do some cheetah chasing to stay in shape.\n-----\nWell nursing stresses you out so I wish luck with sister.",
 "Hi, how are you doing? I'm getting ready to do some cheetah chasing to stay in shape.\n-----\nYeah just want to pick up Nba nfl getting old.",
 "Hi, how are you doing? I'm getting ready to do some cheetah chasing to stay in shape.\n-----\nI really like Celine Dion. What about you?",
 "Hi, how are you doing? I'm getting ready to do some cheetah chasing to stay in shape.\n-----\nNo. I live near farms.",
 "Hi, how are you doing? I'm getting ready to do some cheetah chasing to stay in shape.\n-----\nI wish I had a dau

In [27]:
# Example of target with class probabilities
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
print(input)
target = torch.randn(3, 5).softmax(dim=1)
print(target)
output = loss(input, target)
output.backward()

tensor([[-0.0754,  1.3154,  1.8942, -1.5395, -1.9320],
        [ 0.6195,  0.4196,  1.4826, -1.3767, -1.4139],
        [-2.5223,  0.6830, -2.5059,  0.1596, -0.1500]], requires_grad=True)
tensor([[0.1011, 0.0790, 0.1870, 0.4164, 0.2165],
        [0.0636, 0.5943, 0.1131, 0.1221, 0.1070],
        [0.5366, 0.0769, 0.0543, 0.3117, 0.0204]])


In [8]:
# wandb.init(
#     project="bloom-personachat",
#     config={
#         "num_samples": NUM_SAMPLES,
#         "batch_size": BATCH_SIZE,
#         "learning_rate": LR,
#         "weight_decay": WEIGHT_DECAY,
#         "num_prefix_tokens": NUM_PREFIX_TOKENS,
#         "model_name": MODEL_NAME,
#         "seed": SEED,
#     }
# )
loss_hist = []
print('wandb initialized\n')

for batch in tqdm(train_dataloader):
    batch = {k: v.to(DEVICE) for k, v in batch.items()}

    model.train()
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    print(f"Train Loss: {loss}")
    loss_hist.append(loss)

    wandb.log({"Train Loss": loss})

wandb initialized



  0%|          | 1/250 [00:33<2:18:05, 33.28s/it]

Train Loss: 5.8812384605407715


  1%|          | 2/250 [01:05<2:14:39, 32.58s/it]

Train Loss: 6.158038139343262


...

Train Loss: 2.169942617416382


 80%|███████▉  | 199/250 [1:43:19<26:00, 30.59s/it]

Train Loss: 2.2558305263519287


 80%|███████▉  | 199/250 [1:43:21<26:29, 31.16s/it]


KeyboardInterrupt: 

In [16]:
%%time
outputs = model.generate(
    tokenizer(['something'], return_tensors='pt')['input_ids'],
    temperature=1.0,
    do_sample=True,
    top_k=10,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=250,
)

CPU times: user 28min 16s, sys: 7.11 s, total: 28min 23s
Wall time: 1min 57s


In [20]:
print(tokenizer.decode(outputs[0]))

something else, it shouldnt be able to recognize the word "engineer", as it does not exist in its lexicon.
Is it possible to do this?
Thanks in advance,
Alex.

A:

The problem is that the "engineer" word isn't in your lexicon, therefore, when you run the model and the model sees "engineer", it doesn't understand it. You can try the following code to make it understand it (just replace "engineer" with the word you want):
from __future__ import print_function

from gensim import models
from sklearn.externals.joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler
from gensim.models import Word2Vec, SkipGramModel
from gensim.models.word2vec.utils import pad_sequences

import numpy as np
from scipy.sparse import csr_matrix 
from sklearn import preprocessing
from sklearn.feature_selection import f_classif
from sklearn.cross_validation import KfoldCV as sklearnCV
from gensim.models import Word2Vec, SkipGramModel
from gensim.models.word2vec.utils import pad_sequences


Try to talk with the trained model! Submit an empty input to stop the execution.


__Note__: In this example, we the whole dialogue as a prefix when generating each new replica. In the future, we will support a faster "interactive" dialogue mode, so generating a new replica will be able to reuse inference caches from the previous replica.

In [ ]:
MAX_TOKENS = 16
TOP_K = 100
TEMPERATURE = 0.6
dialog = ""

while True:
    user_phrase = input()
    if len(user_phrase) == 0:
        break
    dialog += f"{user_phrase}\n-----\n"
    inputs = tokenizer([dialog], return_tensors='pt')['input_ids']
    outputs = model.generate(
        inputs,
        temperature=TEMPERATURE,
        do_sample=True,
        top_k=TOP_K,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=MAX_TOKENS,
    )
    bloom_answer = tokenizer.batch_decode(outputs)[0]
    bloom_answer = bloom_answer[len(dialog):].split("\n")[0]
    print(bloom_answer)
    dialog += f"{bloom_answer}\n-----\n"